# Slide Example RLWE
This notebook covers the example from the slides i.e, without using any random numbers.

In [1]:
import numpy as np

# Check for correct numpy version
np_version = np.version.version
if np_version[:-2] != '1.24':
    print(f"WARNING: NumPy version! Please use NumPy version 1.24.x for the best experience. Use of an incorrect numpy version could cause the printing of polynomials to work differently. As the expected order of polynomial coefficients was changed. In this implementation we expect the order to be highest to lowest degree.")

### Utility Functions

In [2]:
def poly_to_str(p) -> str:
    """Converts polynomial to string.  
    Assumes all coefficients are postive integers"""
    # Reverse the order an convert to int
    p2 = [int(el) for el in p[::-1]]
    p_stack = [""] * len(p2)
    
    # Handle constant term and x term
    p_stack[0] = str(p2[0]) if p2[0] != 0 else ""
    p_stack[1] = f"{p2[1]}x" if (p2[1] > 1) else ("x" if p2[1] == 1 else "")

    # Handle x^n terms
    for i in range(2, len(p2)):
        if p2[i] > 1:
            p_stack[i] = f"{p2[i]}x^{i}"
        elif p2[i] == 1:
            p_stack[i] = f"x^{i}"

    final_str = ""
    # Reverse back
    p_stack = p_stack[::-1]
    # Get final output
    for i in range(len(p_stack)):
        if len(p_stack[i]) > 0:
            final_str += (" + " if i != 0 else "") + p_stack[i]
        
    return final_str

print(poly_to_str([1, 0, 1, 1]))

x^3 + x + 1


### Calculate T1

In [3]:
phi_x= [1, 0, 0, 1]  # x^3 + 1
q = 13  # Order of numbers
max_error = 1

S = [1, 4, 9]  # x^2 + 4x + 9

A1 = [2, 7, 11]
E1 = [1, 0, -1]

def calculate_T(A, E, S, q, phi_x):
    # Multiply A & S
    prod1 = np.polymul(A, S)
    # Take mod q of each number
    prod2 = prod1 % q
    # Reduce the polynomial back to required degree (Or to fit in GF(2))
    prod3 = np.polydiv(prod2, phi_x)[1] % q  # [1] to take the remainder
    # Add the errors
    final_t = np.polyadd(prod3, E) % q  

    return final_t

T1 = calculate_T(A1, E1, S, q, phi_x)
print(f"T1 = {poly_to_str(T1)}")

T1 = 6x^2 + x + 5


### Calculate T2

In [4]:
A2 = [6, 8, 3]
E2 = [-1, 1, 1]

T2 = calculate_T(A2, E2, S, q, phi_x)
print(f"T2 = {poly_to_str(T2)}")

T2 = 10x^2 + x + 9


### Keys

In [5]:
print("Public Key:")
print(f"A1 = {poly_to_str(A1)}\nA2 = {poly_to_str(A2)}\nT1 = {poly_to_str(T1)}\nT2 = {poly_to_str(T2)}\nq = {q}\nphi_x = {poly_to_str(phi_x)}\nmax_error = {max_error}")
print("---------------------------------------------------------")
print("Private Key:")
print(f"S = {poly_to_str(S)}")

Public Key:
A1 = 2x^2 + 7x + 11
A2 = 6x^2 + 8x + 3
T1 = 6x^2 + x + 5
T2 = 10x^2 + x + 9
q = 13
phi_x = x^3 + 1
max_error = 1
---------------------------------------------------------
Private Key:
S = x^2 + 4x + 9


### Encryption

In [6]:
Message = [1, 1, 0]

# Calculate A_new & T_new (Can choose any weighted sum)
A_new = np.polyadd(A1, A2) % q
T_new = np.polyadd(T1, T2) % q

# Add message
new_message = [(q // 2) * m for m in Message]
T_send = np.polyadd(T_new, new_message) % q

# Send
print(f"Sending:\nA_new = {poly_to_str(A_new)}\nT_send = {poly_to_str(T_send)}")

Sending:
A_new = 8x^2 + 2x + 1
T_send = 9x^2 + 8x + 1


### Decryption

In [7]:
# Find T_ideal
T_ideal = calculate_T(A_new, [0]*len(A_new), S, q, phi_x)

# T_send - T_ideal
Message_Draft = T_send - T_ideal

final_message = []
for m_bit in Message_Draft:
    message_bit = ((m_bit + (q//4)) % q) // (q//2)
    final_message.append(int(message_bit))

print(f"Final Message = {final_message}")

Final Message = [1, 1, 0]


### Encyption with extra errors
(Cannot be used with this example as max_error_can_add = 0)

In [11]:
Message = [1, 1, 0]

# Calculate A_new & T_new (Can choose any weighted sum)
A_new = np.polyadd(A1, A2) % q
T_new = np.polyadd(T1, T2) % q

# Add extra errors to T_new
max_error_can_add = (q//4) - (2 * max_error) - 1  # The amount of extra error that can be added
print(f"max_error_can_add = {max_error_can_add}")

max_error_can_add = 0
